<a href="https://colab.research.google.com/github/muke888/DataScience-Competitions/blob/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/Richter_scale_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna -q
!pip install category_encoders -q
import category_encoders as ce
import optuna
from optuna.integration import KerasPruningCallback
import tensorflow as tf #print(tf.__version__)
from tensorflow import keras
#from tensorflow.keras import layers
#import IPython



import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

X = pd.read_csv("https://raw.githubusercontent.com/muke888/DataScience-Competitions/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/train_values.csv")
y = pd.read_csv("https://raw.githubusercontent.com/muke888/DataScience-Competitions/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/train_labels.csv")
test_value = pd.read_csv("https://raw.githubusercontent.com/muke888/DataScience-Competitions/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/test_values.csv")
#submission_format = pd.read_csv("https://raw.githubusercontent.com/muke888/DataScience-Competitions/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/submission_format.csv")

     |████████████████████████████████| 235kB 2.7MB/s 
     |████████████████████████████████| 1.1MB 8.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 112kB 17.5MB/s 
     |████████████████████████████████| 133kB 17.3MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 81kB 2.0MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
X_train.shape[1]

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y.damage_grade, train_size=0.8, random_state=88, stratify=y.damage_grade.values)

X_test = test_value.drop(columns=['building_id'])
X_train=X_train.drop(columns=['building_id'])
X_validation=X_validation.drop(columns=['building_id'])

In [ ]:
#Encoding - BinaryEncoder, CatBoostEncoder
categorical_features=['geo_level_1_id','geo_level_2_id','geo_level_3_id','land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position','plan_configuration', 'legal_ownership_status']
#categorical_features=['land_surface_condition','foundation_type']
encoder = ce.BinaryEncoder(cols=categorical_features, return_df=True)
#encoder = ce.CatBoostEncoder(cols=categorical_features, return_df=True, random_state=888)
#encoder = ce.TargetEncoder(cols=categorical_features, return_df=True)


X_train = encoder.fit_transform(X_train, y_train)
X_validation = encoder.transform(X_validation)
X_test = encoder.transform(X_test)

In [ ]:
#scaler = StandardScaler()
scaler = MinMaxScaler()
X_train[X_train.columns]=scaler.fit_transform(X_train[X_train.columns])
X_validation[X_validation.columns]=scaler.transform(X_validation[X_validation.columns])
X_test[X_test.columns]=scaler.transform(X_test[X_test.columns])
#X_train[['count_floors_pre_eq', 'age', 'area_percentage','height_percentage','count_families']] = scaler.fit_transform(X_train[['count_floors_pre_eq', 'age', 'area_percentage','height_percentage','count_families']])
#X_validation[['count_floors_pre_eq', 'age', 'area_percentage','height_percentage','count_families']] = scaler.transform(X_validation[['count_floors_pre_eq', 'age', 'area_percentage','height_percentage','count_families']])

In [ ]:
#Transforming targets to dummies
classes=pd.get_dummies(y_train.values).columns.values
y_train=pd.get_dummies(y_train.values).values
y_validation=pd.get_dummies(y_validation.values).values

In [ ]:
reconstructed_model = keras.models.load_model("my_model")
predicted_prob = reconstructed_model.predict(X_test)
predicted=reconstructed_model.predict_classes(X_test)
#p2=np.argmax(reconstructed_model.predict(X_test), axis=-1)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(995,input_dim=X_train.shape[1])) #input_shape=(88,)
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.3065638839436802))
model.add(keras.layers.Dense(447))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('selu'))
model.add(keras.layers.Dropout(0.45819766213996094))
#model.add(keras.layers.Dense(128))
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Activation('selu'))
#model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('softmax'))

model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),#,clipvalue=0.5), #1e-3 ,clipvalue=0.5
        #loss = 'categorical_crossentropy',
        loss=keras.losses.CategoricalCrossentropy(),
        #metrics=['acc'])
        metrics=[keras.metrics.CategoricalAccuracy()]) # ['sparse_categorical_accuracy']

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('my_model', monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='auto', save_weights_only=False) #'output/'epoch:{epoch:03d}_val_acc:{val_acc:.4f}'
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto', restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
callbacks_list = [earlystop,reduce_lr]


model.fit(X_train.values,y_train, #y=to_categorical(y_train.values,num_classes=3),
        epochs=500, batch_size=32,
        verbose=1,
        validation_data=(X_validation.values, y_validation),
        callbacks=[checkpoint]
    )

Epoch 1/500
6508/6515 [============================>.] - ETA: 0s - loss: 0.7335 - categorical_accuracy: 0.6631
Epoch 00001: val_categorical_accuracy improved from -inf to 0.68750, saving model to my_model
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model/assets
6515/6515 [==============================] - 48s 7ms/step - loss: 0.7334 - categorical_accuracy: 0.6632 - val_loss: 0.6793 - val_categorical_accuracy: 0.6875
Epoch 2/500
6509/6515 [============================>.] - ETA: 0s - loss: 0.6838 - categorical_accuracy: 0.6851
Epoch 00002: val_categorical_accuracy improved from 0.68750 to 0.69797, saving model to my_model
INFO:tensorflow:Assets written to: my_model/assets
6515/6515 [==============================] - 47s 7ms/step - loss: 0.6838 - categorical_accura

In [ ]:
#https://github.com/optuna/optuna/blob/master/examples/pruning/keras_integration.py
from optuna.integration import TFKerasPruningCallback
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Activation

In [ ]:
def objective(trial):
    keras.backend.clear_session()
    model = Sequential()
    model.add(Dense(units=trial.suggest_int('units1', 88, 1024),input_dim=X_train.shape[1]))
    model.add(BatchNormalization())
    model.add(Activation(activation=trial.suggest_categorical('activation1', ['relu', 'elu','selu'])))
    model.add(Dropout(rate=trial.suggest_float("dropout1", 0.2, 0.5)))
    model.add(Dense(units=trial.suggest_int('units2', 3, 500)))
    model.add(BatchNormalization())
    model.add(Activation(activation=trial.suggest_categorical('activation2', ['relu', 'elu','selu'])))
    model.add(Dropout(rate=trial.suggest_float("dropout2", 0.2, 0.5)))
    model.add(Dense(3))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    # We compile our model with a sampled learning rate.
    #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss = 'categorical_crossentropy',
        metrics=['acc'])
    
    #EarlyStopping = tf.keras.callbacks.EarlyStopping(patience=2)
    #checkpoint = tf.keras.callbacks.ModelCheckpoint('my_model', monitor='val_acc', verbose=1, save_best_only=True, mode='auto', save_weights_only=False)
    model.fit(X_train.values,y_train,
        validation_data=(X_validation.values, y_validation),
        #shuffle=True,
        batch_size=32,
        #batch_size=trial.suggest_categorical('batchsize', [8, 16, 32, 64, 128]),
        epochs=trial.suggest_int('epochs', 50, 500),
        verbose=40,
        callbacks=[TFKerasPruningCallback(trial, "val_accuracy")]
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(X_validation.values, y_validation, verbose=0)
    return score[1]

In [ ]:
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=100)
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

In [ ]:
df_results = study.trials_dataframe()
df_results.to_csv('df_optuna_results.csv')

In [ ]:
df_results

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['units1', 'activation1','dropout1'])